
# node2vec の埋め込み作成を行う
user, item だけだと不安なので、 user,item 間に関してもエッジを貼る(ケースもやる）


In [1]:
import os
import pickle
import random
import sys
import uuid
from pathlib import Path

import implicit
import lightgbm as lgb
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, random
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

# 最大表示列数の指定（ここでは50列を指定）
pd.set_option("display.max_columns", 50)

sys.path.append(os.pardir)
from hydra import compose, initialize

from utils import load_datasets
from utils.embedding import TextEmbedder

with initialize(config_path="../yamls", version_base=None):
    config = compose(config_name="config.yaml")


train_df = pd.read_csv(Path(config.input_path) / "train.csv")
test_df = pd.read_csv(Path(config.input_path) / "test.csv")

sample_submission_df = pd.read_csv(Path(config.input_path) / "sample_submission.csv")
anime_df = pd.read_csv(Path(config.input_path) / "anime.csv")

# 整形
anime_df["genres"] = anime_df["genres"].str.replace(" ", "")

# Merge the train data with the anime meta data
all_df = pd.concat([train_df, test_df])
all_df = all_df.merge(anime_df, on="anime_id", how="left")

In [2]:
import torch
from torch_geometric.nn import Node2Vec
from tqdm.auto import tqdm

OSError: /opt/conda/lib/python3.10/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
all_df.shape

In [14]:
le = LabelEncoder()
all_ids = np.unique(np.concatenate([all_df["user_id"].unique(), all_df["anime_id"].unique()]))
le.fit(all_ids)
all_df["user_label"], user_idx = pd.factorize(all_df["user_id"])
all_df["anime_label"], anime_idx = pd.factorize(all_df["anime_id"])
# userとanimeの番号が別になるようにずらす
all_df["anime_label"] += len(user_idx)
num_nodes = len(user_idx) + len(anime_idx)
edges = all_df[["user_label", "anime_label"]].to_numpy()

In [16]:
model = Node2Vec(
    torch.from_numpy(edges).T,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1,
    q=1,
    sparse=True,
).to(device)
num_workers = 0 if sys.platform.startswith("win") else 4
loader = model.loader(batch_size=128, shuffle=True, num_workers=num_workers)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

for epoch in range(1, 81):
    loss = train()
    print(f"Epoch: {epoch:02d}, Loss: {loss:.4f}")

ImportError: `Node2Vec` requires `torch-cluster`.